In [2]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

Using TensorFlow backend.
C:\Users\m.kengesbay\Anaconda3\new\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\m.kengesbay\Anaconda3\new\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\m.kengesbay\Anaconda3\new\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\m.kengesbay\Anaconda3\new\lib\site-packag

In [3]:
data = pd.read_excel('full_text_no_num_60k_40cat.xlsx') #full_text_no_num_60k

In [4]:
# lets take 80% data as training and remaining 20% for test.
train_size = int(len(data) * .8)
 
train_posts = data['text'][:train_size]
train_tags = data['tags'][:train_size]
#train_files_names = data['filename'][:train_size]
 
test_posts = data['text'][train_size:]
test_tags = data['tags'][train_size:]
#test_files_names = data['filename'][train_size:]

In [8]:
# 20 news groups
num_labels = 41
vocab_size = 10000
batch_size = 1000
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_posts)
 
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [15]:
train_tags

0              Рынки
1            Техника
2           Криминал
3        Бывший СССР
4              Рынки
            ...     
47195    Бывший СССР
47196    Бывший СССР
47197        События
47198       Политика
47199      Экономика
Name: tags, Length: 47200, dtype: object

In [9]:
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [14]:
y_test.shape

(11800, 41)

In [101]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy',f1_m,precision_m, recall_m])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    verbose=1,
                    validation_split=0.2)

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 512)               20480512  
_________________________________________________________________
activation_67 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_45 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_68 (Dense)             (None, 512)               262656    
_________________________________________________________________
activation_68 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_46 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_69 (Dense)             (None, 41)              

In [102]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('accuracy:', score[1])
 
    
text_labels = encoder.classes_
 
for i in range(50):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction[0])]
    
    #print(prediction)
    ind = np.argpartition(prediction[0], -3)[-3:]
    ind = ind[np.argsort(prediction[0][ind])]
    #print(ind)
    
    #rint(prediction[0][ind], prediction[0][ind][1] )
    
    print('Actual label:' + str(test_tags.iloc[i]))
    #print("Predicted label: " + str(predicted_label))
    print("Predicted label 1:" +str(text_labels[np.array(ind[2])]),";","Predicted label 2: " +str(text_labels[np.array(ind[1])]),";","Predicted label 3: " + str(text_labels[np.array(ind[0])]) )
    print("Prediction 1: " +str(prediction[0][ind][2]),";","Prediction 2: " +str(prediction[0][ind][1]),";","Prediction 3: " + str(prediction[0][ind][0]) )
    #print('Text:' + str(test_posts.iloc[i]))
    print()
    

11800/11800 [==============================] - 27s 2ms/step
accuracy: 0.6989830732345581
Actual label:Экономика
Predicted label 1:Экономика ; Predicted label 2: Деньги ; Predicted label 3: Рынки
Prediction 1: 0.9960963 ; Prediction 2: 0.0038464875 ; Prediction 3: 2.7374756e-05

Actual label:Следствие и суд
Predicted label 1:Полиция и спецслужбы ; Predicted label 2: Криминал ; Predicted label 3: Россия
Prediction 1: 0.9777123 ; Prediction 2: 0.017907476 ; Prediction 3: 0.0020635747

Actual label:Экономика
Predicted label 1:Экономика ; Predicted label 2: Рынки ; Predicted label 3: Деньги
Prediction 1: 0.9995895 ; Prediction 2: 0.00040617216 ; Prediction 3: 4.3119276e-06

Actual label:Дом
Predicted label 1:Дом ; Predicted label 2: Россия ; Predicted label 3: Деловой климат
Prediction 1: 0.9376777 ; Prediction 2: 0.044585668 ; Prediction 3: 0.0033463559

Actual label:Закавказье
Predicted label 1:Закавказье ; Predicted label 2: Бывший СССР ; Predicted label 3: Мир
Prediction 1: 1.0 ; Predic